In [ ]:
from google.colab import drive, files
drive.mount('/content/drive/')

%cd /content/drive/MyDrive/College/2021Spring/Thesis/Experiments/CNN/
%ls

Mounted at /content/drive/
/content/drive/MyDrive/College/2021Spring/Thesis/Experiments/CNN
CNN.ipynb  X.npy  Y.npy


In [ ]:
import keras
from keras.datasets import mnist
import numpy as np

#load mnist dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data() 
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# Want binary regression problem so choose to differentiate between 0 and 1
# X_train = X_train[y_train < 2, ]
# y_train = 
X_zeros = X_train[y_train == 0]
X_ones = X_train[y_train == 1]
print(X_zeros.shape)
print(X_ones.shape) 

X_sub = np.concatenate((X_zeros[:30], X_ones[:30]), axis = 0)
y_sub = np.repeat([0, 1], 30, axis = 0)
print(y_sub)
shuff = np.random.choice(60, 60, replace = False)
X_sub = X_sub[shuff]
y_sub = y_sub[shuff]
print(y_sub)

print(X_sub.shape)
X_sub = X_sub.astype('float32')
X_sub /= 255
print(np.max(X_sub))

# Save subsets of X and Y
np.save("X.npy", X_sub)
np.save("Y.npy", y_sub)

(5923, 28, 28, 1)
(6742, 28, 28, 1)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 1 0 0 1 1 1 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 0 1 0 0
 0 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 1 1 0]
(60, 28, 28, 1)
1.0


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
model = Sequential()
conv_layer = Conv2D(32, kernel_size=(11, 11),
                 input_shape=input_shape, use_bias = False)
conv_layer.trainable = True
model.add(conv_layer)

In [ ]:
# Test Keras Flatten behavior
# unrolls the values beginning at the last dimension
out = model.predict(X_sub)
print(out.shape)
print(out[6, 2, 3, 4])
print(out[6, 4, 3, 2])

tmp_model = Sequential()
tmp_model.add(Flatten(data_format = "channels_last"))
tmp_out = tmp_model.predict(out)
print(tmp_out.shape)
print(tmp_out[6, (2*18 + 3)*32 + 4])
print(tmp_out[6, (4*18 + 3)*32 + 2])

(60, 18, 18, 32)
0.078282274
-0.17071404
(60, 10368)
0.078282274
-0.17071404


In [ ]:
# Verify that reshaping convolutional filters is as expected
filters = model.layers[0].get_weights()[0]
print(filters.shape)
print(filters[5, 6, 0, 4])
reshaped = filters.reshape(11, 11, -1)
print(reshaped.shape)
print(reshaped[5, 6, 4])

(11, 11, 1, 32)
0.010239787
(11, 11, 32)
0.010239787


In [ ]:

# Test Keras convolution behavior (flipped or not) - NOT FLIPPED
print("Test flip")
test = X_sub[0, :11, :11, 0]
# print(test)
print(out[0, 0, 0, 0])
one_filter = reshaped[:, :, 0]
print(one_filter.shape)
# not flipped
print(np.sum(np.multiply(test, one_filter)))
# flipped
print(np.sum(np.multiply(test, one_filter[::-1, ::-1])))

Test flip
-0.025586884
(11, 11)
-0.025586884
-0.0017385734


In [ ]:
# Set seeds
seed_value = 1
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)

# Build model
model = Sequential()
conv_layer = Conv2D(32, kernel_size=(11, 11),
                 input_shape=input_shape, use_bias = False)
conv_layer.trainable = True
model.add(conv_layer)

model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Flatten(data_format = "channels_last"))

dense_layer = Dense(1, activation='relu')
dense_layer.trainable = True
model.add(dense_layer)

In [ ]:
# Prediction dimensions
out = model.predict(X_sub)
print(out.shape)

(60, 1)
